# Understanding Tesco Mobile reviews on Trustpilot

The scope of this project includes:
* scraping reviews from TrustPilot
* training an NLP model to predict the sentiment of a new review 
* categorise reviews and track their scores over time 

In [1]:
# import basic modules
import pandas as pd
import numpy as np 

In [6]:
# import modules
import spacy
from spacy.matcher import PhraseMatcher

ModuleNotFoundError: No module named 'spacy'

## Step 3: Categorise reviews based on their content

In [ ]:
# load the dataset
data = pd.read_csv(
    '../data/tp_reviews_20180915_20200830.csv', 
    parse_dates=['published_date','updated_date','reported_date'])

# inspect
data.head()

In [ ]:
#
df = data.copy()

# drop columns that we won't need 
to_drop = ['updated_date','reported_date']
df.drop(columns=to_drop, axis=1, inplace=True)

# rename published_date
df.rename(columns={'published_date':'published_at'}, inplace=True)

# inspect 
df.head()

In [ ]:
# let's create some initial categories 
categories_dict = {
    'customer service': ['customer service','customer services','customer care','staff','customer experience'],
    'coverage': ['coverage','signal','no service'],
    'internet': ['speed','4G','buffer','internet','data allowance','data'],
    'billing': ['upfront cost','bill','payment','fees'],
    'sim': ['sim only','SIM'],
    'payg': ['pay as you go', 'topup','top up', 'top-up', 'rocket pack', 'rocketpack','rocketpacks','e-voucher','payg'],
    'insurance': ['insurance'],
    'clubcard': ['clubcard','clubcard plus','club card']}

In [ ]:
# create a lookup table that we can reference later
categories = []
words = []

for cat, terms in categories_dict.items():
    for word in terms:
        words.append(word)
        categories.append(cat)
        
# 
categories = pd.Series(categories, name='cat')
words = pd.Series(words, name='term')

# join 
categories_df = pd.DataFrame(words).join(categories)

# inspect
categories_df.head()

In [ ]:
# turn the categories into a list because I think it's what spacy needs 
cat_list = categories_df['term'].to_list()